In [ ]:
!pip install "shapely<2.0.0"
!pip install google-cloud-aiplatform langchain --upgrade
#! pip install google-cloud-aiplatform langchain==0.0.323 chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# LangChain

In [ ]:
import vertexai
import langchain
from langchain.llms import VertexAI
from google.cloud import aiplatform

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

In [ ]:
llm = VertexAI(
    model_name="code-bison@002",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

print(llm("""Write me a function to reverse the bits of an
             integer in Objective-C
          """))

# Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Context: You write SQL Queries based on natural language inputs.
    Given the following database Schema: {schema}

    Write a SQL SELECT query based on the following question: {question}.
    Only return the SQL statement
    Make sure you put semicolons after each line
    """
)

schema = """
TABLE Owners (
  OwnerID STRING(36) NOT NULL,
  OwnerName STRING(MAX) NOT NULL,
  LastName STRING(MAX),
) PRIMARY KEY(OwnerID);

TABLE Pets (
  OwnerID STRING(36) NOT NULL,
  PetID STRING(MAX) NOT NULL,
  PetType STRING(MAX) NOT NULL,
  PetName STRING(MAX) NOT NULL,
  Breed STRING(MAX) NOT NULL,
  DOB DATE,
) PRIMARY KEY(OwnerID, PetID),
  INTERLEAVE IN PARENT Owners ON DELETE CASCADE;
"""

In [ ]:
print(llm(prompt_template.format(schema=schema,
                                 question="How many owners are there")))

In [ ]:
print(llm(prompt_template.format(schema=schema,
                                 question="""Show me the number of pets
                                 for each owner. Include the name of each owner""")))

In [ ]:
print(llm(prompt_template.format(schema=schema,
                                 question="Show me the dogs sorted by age")))

In [ ]:
print(llm(prompt_template.format(schema=schema,
                                 question="""Show me the average age of the pets by type.
                                 Round the average age to the year""")))